In [ ]:
import run_problem_sim
from run_problem_sim import *

In [ ]:
parrallel = True
#Change this depending on the type of sim

# This is for chess
# organism_creator = make_organism_generator((384,), 1)

# This is for cars
organism_creator = make_organism_generator((1269,), 1)

if parrallel:
    scoring_function = scoring_function_parallel
else:
    scoring_function = lambda organism_1, organism_2 : sim.simulate_and_evaluate_organism(organism_1, organism_2, num_sims=1)
ecosystem = Ecosystem(organism_creator, scoring_function, population_size=40, holdout=0.1, mating=False)

generations = 10
print("Running generations")
run_generations(ecosystem, generations, parallel=True)